### Importação das bibliotecas necessárias para a raspagem de dados.

In [ ]:
# pip install selenium
# pip install webdriver-manager
# pip install BeautifulSoup4

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
from time import sleep
# import requests
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
# options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

## Acessando o site Open Street Map e capturando as rotas pendentes e as rotas finalizadas.

In [ ]:
# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

In [ ]:
#list_finished_routes
#list_pending_routes

## Acessando a telas para efetuar os downloads.

In [ ]:
## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(3)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
navegador.close()

## Executando de uma só vez

In [60]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from time import sleep
from bs4 import BeautifulSoup
servico = Service(ChromeDriverManager().install())

# from selenium.webdriver.chrome.options import Options
# import requests
# options = Options()
# options.add_argument('--headless') #--> Parâmetro adicional onde é possivel realizar o webscrapping sem abrir o navegador.
#options.add_argument('window-size=400,800') #--> Parâmetro adicional onde é possivel escolher o tamanho da tela aberta no navegador. Nesse exemplo o tamanho 400x800 é como se estivesse aberto em um celular.

URL_OPEN_STREET_MAP_TRACES = 'https://www.openstreetmap.org/traces' #--> Página do OpenstreetMap onde estão localizadas as rotas para download.
PREFIX_URL_DOWNLOAD = 'https://www.openstreetmap.org' #--> Página do principal do OpenstreetMap. Esta variável será utilizada para montar a URL das páginas de download.

# navegador = webdriver.Chrome(service=servico, options=options) #--> Aplicando as opções acima mencionadas no navegador.

navegador = webdriver.Chrome(service=servico)
navegador.get(URL_OPEN_STREET_MAP_TRACES)

page_content = navegador.page_source
site = BeautifulSoup(page_content, 'html.parser')
routes = site.findAll('tr')

list_pending_routes = []
list_finished_routes = []
list_routes = []

for route in routes:
    if route.find('span', attrs={'class': 'text-danger'}):
        pending_routes = route.find('span', attrs={'class': 'text-danger'})    
        pending_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # list_pending_routes.append([PREFIX_URL_DOWNLOAD+pending_link_routes['href']])
        # print (f"{pending_routes.text}: {PREFIX_URL_DOWNLOAD+pending_link_routes['href']}")

    else:         
        finished_link_routes = route.find('a')        
        list_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # list_finished_routes.append([PREFIX_URL_DOWNLOAD+finished_link_routes['href']])
        # print(f"FINISHED: {PREFIX_URL_DOWNLOAD+finished_link_routes['href']}")
#navegador.close()

#len(list_pending_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes
#len(list_finished_routes) #--> Verificando a quantidade de registros na lista de rotas pendentes

## Quando usa [requests.get] estamos usando o BeautifulSoup - e quando usamos só o [.get] estamos usando o selenium
#navegador = webdriver.Chrome(service=servico)
for list_route in list_routes:
    sleep(3)
    url = list_route[0]
    navegador.get(url)  #--> Exemplo onde usamos o Selenium somente com o [.get]
    navegador.find_element('xpath','//*[@id="content"]/div[2]/div/table/tbody/tr[1]/td/a').click()
    # Capturar a informação da descrição e atribuir para uma variável
    # verificar se o arquivo foi baixado com a extensão ['.crdownload'] remover a extensão e renomear o com a informação [da descrição] + nome do arquivo
    # Se o arquivo não tiver a extensão errada, somente renomear o arquivo com a informação [da descrição] + nome do arquivo


navegador.close()

## Movendo os arquivos da pasta Download para o MINIO_DATA_LAKE

In [61]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
DOWNLOADS = '/home/thiago/Downloads/'
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)
#print(minioclient.list_buckets())
#print("Quantidade de Buckets", len(minioclient.list_buckets()))

In [62]:
files_to_rename = [arquivo for arquivo in os.listdir(DOWNLOADS) if arquivo.endswith(".crdownload")]
for file_name in files_to_rename:
        new_name = file_name.replace(".crdownload", "")
        os.rename(os.path.join(DOWNLOADS, file_name), os.path.join(DOWNLOADS, new_name))

In [63]:
## Aqui os arquivos com a extensão .gpx saem da pasta Download para o bucket MinIO
files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
for name_file in files_to_move:
    local_path = os.path.join(DOWNLOADS, name_file)    
    if os.path.isfile(local_path):
        try:
            minioclient.fput_object(BRONZE_LAYER, name_file, local_path)
            print(f"Arquivo {name_file} enviado com sucesso para o bucket.")
            os.remove(local_path) # --> Após o envio bem sucedido para o bucket o arquivo é excluído da pasta download
        except S3Error as e:
            print(f"Erro ao enviar o arquivo: {name_file} -> Erro: {e}")          

Arquivo 9564403.gpx enviado com sucesso para o bucket.
Arquivo 9564407.gpx enviado com sucesso para o bucket.
Arquivo 9564410.gpx enviado com sucesso para o bucket.
Arquivo 9564402.gpx enviado com sucesso para o bucket.
Arquivo 9564395.gpx enviado com sucesso para o bucket.
Arquivo 9564411.gpx enviado com sucesso para o bucket.
Arquivo 9564414.gpx enviado com sucesso para o bucket.
Arquivo 9564409.gpx enviado com sucesso para o bucket.
Arquivo 9564404.gpx enviado com sucesso para o bucket.
Arquivo 9564405.gpx enviado com sucesso para o bucket.
Arquivo 9564400.gpx enviado com sucesso para o bucket.
Arquivo 9564415.gpx enviado com sucesso para o bucket.
Arquivo 9564394.gpx enviado com sucesso para o bucket.
Arquivo 9564397.gpx enviado com sucesso para o bucket.
Arquivo 9564406.gpx enviado com sucesso para o bucket.
Arquivo 9564408.gpx enviado com sucesso para o bucket.
Arquivo 9564399.gpx enviado com sucesso para o bucket.
Arquivo 9564412.gpx enviado com sucesso para o bucket.
Arquivo 95

# VERIFICAR A FORMA DE LER OS ARQUIVOS QUE ESTÃO NA PASTA DE DOWNLOAD - CONVERTER PARA CSV OU PARQUET E APÓS ESSA CONVERSÃO LEVAR PARA O MINIO

## Lendo arquivos do Bucket

In [64]:
# pip install minio
import os
from minio import Minio
from minio.error import S3Error
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
from xml.dom import minidom


DOWNLOADS = '/home/thiago/Downloads/'
BRONZE_LAYER = 'bronze'

minioclient = Minio('localhost:9000',
    access_key='minioadmin',
    secret_key='minioadmin',
    secure=False)

In [65]:
info_rota = []

arquivos_rotas_gpx = [arquivo_gpx for arquivo_gpx in minioclient.list_objects(BRONZE_LAYER) if arquivo_gpx.object_name.endswith(".gpx")]

In [66]:
for arquivo_rota_gpx in arquivos_rotas_gpx:    
    try:
        blob_rota_gpx = minioclient.get_object(BRONZE_LAYER, arquivo_rota_gpx.object_name)
        rota_gpx = blob_rota_gpx.data
        rota_gpx = rota_gpx.decode("ISO-8859-1")
        try:
            gpx = gpxpy.parse(rota_gpx)
        
            for track in gpx.tracks:
                for segment in track.segments:
                    for point in segment.points:
                        info_rota.append({
                        'latitude': point.latitude,
                        'longitude': point.longitude,
                        'elevacao' : point.elevation,
                        'time_point' : point.time})
    
        except gpxpy.gpx.GPXXMLSyntaxException as e:
            print(f"Erro de sintaxe XML no arquivo {arquivo_rota_gpx.object_name}: {e}")
    except S3Error as e:
        print(f"Erro ao acessar o objeto: {e}")




                

    

Erro de sintaxe XML no arquivo 9557190.gpx: Error parsing XML: unclosed token: line 1929, column 4


In [67]:
info_rota

[{'latitude': 22.9912578,
  'longitude': 120.20585899999999,
  'elevacao': 32.746,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 54, 200000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 22.991258,
  'longitude': 120.20585949999999,
  'elevacao': 32.804,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 54, 300000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 22.9912644,
  'longitude': 120.2058768,
  'elevacao': 34.08,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 56, 300000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 22.9912645,
  'longitude': 120.2058773,
  'elevacao': 34.077,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 56, 400000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 22.9912647,
  'longitude': 120.2058776,
  'elevacao': 34.075,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 56, 500000, tzinfo=SimpleTZ("Z"))},
 {'latitude': 22.9912648,
  'longitude': 120.20587809999999,
  'elevacao': 34.082,
  'time_point': datetime.datetime(2023, 8, 20, 0, 57, 56, 600000, tzinfo=Si

In [38]:

    
type(rota_gpx)

str

## Transformando os arquivos .gpx em um dataframe e CSV e iniciando com PySpark

In [ ]:
# pip install pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('tcc-project').getOrCreate()

In [10]:
# pip install gpxpy
from io import BytesIO
import gpxpy 
import gpxpy.gpx 
import pandas as pd
import xml.etree.ElementTree as ET

In [23]:
with open('9557214.gpx', 'r', encoding='utf-8') as gpx_file:    
    gpx = gpxpy.parse(gpx_file)


In [36]:
type(gpx)

gpxpy.gpx.GPX

In [ ]:
# Para retornar a quantidade de pontos (lat+lon+ele) em um arquivo arquivo '.gpx' podemos usar get
# Documentação: https://pypi.org/project/gpxpy/
gpx.get_track_points_no()

In [ ]:
# Para retornar a faixa de altitude, a fim de se obter extremos de maior elevação e menor elevação no trajeto percorrido - os valores apresentados são em metros acima do nível do mar
gpx.get_elevation_extremes()

In [ ]:
# Não endendi para que isso serve
gpx.get_uphill_downhill()

In [ ]:
# Obtendo o nome do criador da rota
creator = gpx.creator
creator

In [ ]:
# Para exibir o conteudo do arquivo .gpx em formato xml
print(gpx.to_xml()[:1000])

In [ ]:
# É possivel verificar quantas rotas/trilhas nosso arquivo .gpx possui
len(gpx.tracks)

In [ ]:
# Como no nosso exemplo acima só temos 1 rota/trilha efetuada, podemos acessar por meio de indice no python, 
# Nesse caso se passa o valor [0] para ter uma precisão exata da rota em que estamos trabalhando
gpx.tracks[0]

In [ ]:
# Agora vamos acessar os segmentos da nossa rota/trilha
gpx.tracks[0].segments[0]

In [ ]:
# Agora podemos acessar os pontos de dados individuais dentro do nosso gpx acessando a matriz de pontos
gpx.tracks[0].segments[0].points[:10]

In [ ]:
info_rota = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            info_rota.append({
                'latitude': point.latitude,
                'longitude': point.longitude,
                'elevacao' : point.elevation,
                'time_point' : point.time
            })


In [ ]:
info_rota[:3]

In [ ]:
info_rota_df = pd.DataFrame(info_rota)
info_rota_df.head()

In [ ]:
info_rota_df.to_csv('info_rota.csv', index=False)

In [ ]:
# <creator> contem o nome do cara que está se deslocando
# existe uma <trkpt> acredito q seja um track point
# <name> contem uma da e hora. Acredito que seja a hora do envio do arquivo gpx
# <trkpt> é um bloco com lat - lon - internamente tem um <ele> de elevação e um <time> que acredito ser o horario que o cliente estava no ponto lat+long+ele

In [ ]:
## Aqui os arquivos saem da pasta download para a pasta DATALAKE_STAGE

# files_to_move = [files for files in os.listdir(DOWNLOADS) if files.endswith(".gpx")]
# for name_files in files_to_move:
#     origin_path = os.path.join(DOWNLOADS, name_files)
#     destiny_path = os.path.join(DATALAKE_STAGE, name_files)
#     os.rename(origin_path, destiny_path)
#     print(f"Arquivo {name_files} movido para {destiny_path}")

In [ ]:
## Aqui os arquivos saem da pasta DATALAKE_STAGE para o bucket MinIO

# for item in os.listdir(DATALAKE_STAGE):
#     local_path = os.path.join(DATALAKE_STAGE, item)
#     print(local_path)
# 
#     if os.path.isfile(local_path):
#         try:
#             minioclient.fput_object(DATA_LAKE, item, local_path)
#             print(f"Arquivo {item} enviado com sucesso para o bucket.")
#         except S3Error as err:
#             print(f"Erro ao enviar o arquivo {item}: {err}")

In [ ]:
# ## DADOS PARA INSERIR NA TABELA DE CONTROLE USANDO O BEAUTIFULSOAP##
# navegador = requests.get('https://www.openstreetmap.org/user/sunnypilot/traces/9255974')
# site = BeautifulSoup(navegador.text, 'html.parser')
# data_trace = site.find('div', attrs={'class': 'content-body'})
# table_trace = data_trace.find('table')
# 
# filename_row = table_trace.find('th', string='Filename:').parent
# owner_row = table_trace.find('th', string='Owner:').parent
# uploaded_row = table_trace.find('th', string='Uploaded:').parent
# 
# filename = filename_row.find('td').text.strip().replace('(download)','').strip()
# owner = owner_row.find('td').text.strip()
# uploaded = uploaded_row.find('td').text.strip()